# Fine-tune Gemma 2B model
--
## Assignment #4 - Miguel Herrera

## For this assignment, I'm using a different dataset : **bhoopesh/llama3_medical_dataset** from huggingface

---
In this workbook, I'll reuse the code shared in class. However, it's important to note that during he adaptation of the code to train the Gemma 2B model with a different dataset, I encountered several technical difficulties which I was able to resolve.  

Some of those limitations and it's resolution is indicated in the notes below.

---

### Set up

---
We begin by installing and upgrading necessary packages. Restart the kernel after executing the cell below for the first time.

---

In [ ]:
!pip install --upgrade sagemaker datasets --quiet

## Deploy Pre-trained Model

---

Selecting the new model "Gemma 2b"

In [80]:
model_id, model_version = "huggingface-llm-gemma-2b", "1.3.0"

In [81]:
from sagemaker.jumpstart.model import JumpStartModel

pretrained_model = JumpStartModel(model_id=model_id, model_version=model_version)
pretrained_predictor = pretrained_model.deploy(instance_type="ml.g5.xlarge", accept_eula=True)

No instance type selected for inference hosting endpoint. Defaulting to ml.g5.xlarge.
INFO:sagemaker.jumpstart:No instance type selected for inference hosting endpoint. Defaulting to ml.g5.xlarge.
INFO:sagemaker:Creating model with name: hf-llm-gemma-2b-2024-10-21-05-07-08-033
INFO:sagemaker:Creating endpoint-config with name hf-llm-gemma-2b-2024-10-21-05-07-08-037
INFO:sagemaker:Creating endpoint with name hf-llm-gemma-2b-2024-10-21-05-07-08-037


----------!

In [82]:
import sagemaker
# execution role for the endpoint
role = sagemaker.get_execution_role()

# sagemaker session for interacting with different AWS APIs
sess = sagemaker.session.Session()

# Region
region_name = sess._region_name

print(f"sagemaker role arn: {role}")
print(f"sagemaker session region: {region_name}")

sagemaker role arn: arn:aws:iam::160885283791:role/service-role/AmazonSageMaker-ExecutionRole-20241012T110743
sagemaker session region: us-west-2


In [84]:
from sagemaker.predictor import Predictor

endpoint_name = pretrained_predictor.endpoint_name

llm = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.JSONDeserializer(),
)

## Invoke the endpoint

---
Next, I invoke the endpoint  and test with some sample queries.

---

In [85]:
def print_response(payload, response):
    print(payload["inputs"])
    
    print(f"> {response.get('generated_text')}")
    
    print("\n==================================\n")

In [86]:
payload = {
    "inputs": "What's the most popular name for boys?",
    "temperature": 0.9,
    "top_p": 0.9,
    "max_tokens": 500,
}
try:
    response = llm.predict(
        payload
    )
    print_response(payload, response)
except Exception as e:
    print(e)

What's the most popular name for boys?
'list' object has no attribute 'get'


In [87]:
print(response[0]['generated_text'])

What's the most popular name for boys? Find out the boy's names that are popular these days and learn how they rate on our national and state charts. Browse the charts side-by-side to compare data across the many variations and historical periods - use the Venn diagram to compare top names for girls and boys as well!
In October 2016, Roma = Female.


## Dataset preparation for fine-tuning

---

For this assignment, I selected the medical dataset [Llama 3 Medical Dataset](hhttps://huggingface.co/datasets/bhoopesh/llama3_medical_dataset), mainly because it's similar structure with the model used for demo purposes in class. Llama 3 Medica dataset contains 2,000 entries designed for training and fine-tuning Language Learning Models (LLMs) in the medical domian. 

---

## Trying to solve issues with the fsspec library giving error message

---

As mentioned, when adapting the code, I encountered issues with the **fsspec** library.  The version installed returned a "Nothing" value in the version, which did not let me uninstall it or upgrade it using PIP, therefore a more "drastic" measure was taken and I deleted the library from the folder completely as shown below.  Then I was able to re-install this library and was able to get a version number. 

---

In [14]:
!pip install --force-reinstall fsspec==2023.6.0

1016.95s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Using cached fsspec-2023.6.0-py3-none-any.whl.metadata (6.7 kB)
Using cached fsspec-2023.6.0-py3-none-any.whl (163 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec None
error: uninstall-no-record-file

× Cannot uninstall fsspec None
╰─> The package's contents are unknown: no RECORD file was found for fsspec.

hint: You might be able to recover from this via: pip install --force-reinstall --no-deps fsspec==2023.6.0


### I was getting an error message with the **fsspec** library, so had to delete manually all the references and install again

In [18]:
!rm -rf /opt/conda/lib/python3.11/site-packages/fsspec
!rm -rf /opt/conda/lib/python3.11/site-packages/fsspec-*.dist-info

1306.05s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
1311.27s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


---

Installing **fsspec** library again

---

In [ ]:
!pip install fsspec

### After resolving the issue above, I was able to load the finbro dataset. 

---

For this assignment, I'm using the Llama 3 Medical dataset. 

In [42]:
from datasets import load_dataset

#finbro_dataset = load_dataset("bhoopesh/llama3_medical_dataset", split="data")
medical_dataset = load_dataset("bhoopesh/llama3_medical_dataset", split="data") ## new dataset

# For demonstration purposes of this tutorial, we train our model with 5% of the whole dataset. The test data is used to evaluate at the end.
train_and_test_dataset = medical_dataset.train_test_split(test_size=0.95)  # train_size= 0.9, test_size=0.1

# Dumping the training data to a local file to be used for training.
train_and_test_dataset["train"].to_json("train.jsonl")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

290158

In [43]:
train_and_test_dataset['train']

Dataset({
    features: ['input', 'output', 'instruction', 'prompt'],
    num_rows: 100
})

In [44]:
train_and_test_dataset["train"][8]

{'input': 'What are the symptoms of Chronic Eosinophilic Leukemia ?',
 'output': 'Signs and symptoms of chronic eosinophilic leukemia include fever and feeling very tired. Chronic eosinophilic leukemia may not cause early signs or symptoms. It may be found during a routine blood test. Signs and symptoms may be caused by chronic eosinophilic leukemia or by other conditions. Check with your doctor if you have any of the following:         -  Fever.    - Feeling very tired.    - Cough.    - Swelling under the skin around the eyes and lips, in the throat, or on the hands and feet.    - Muscle pain.    - Itching.    -  Diarrhea.',
 'instruction': 'Answer the question truthfully, you are a medical professional.',
 'prompt': 'system Answer the question truthfully, you are a medical professional. user This is the question: What are the symptoms of Chronic Eosinophilic Leukemia ? assistant Signs and symptoms of chronic eosinophilic leukemia include fever and feeling very tired. Chronic eosinoph

---
Next, I create a prompt template for using the data in an instruction / input format for the training job (since we are instruction fine-tuning the model in this example), and also for inferencing the deployed endpoint.

---

In [45]:
import json

template = {
    "prompt": "### Input:\n{input}\n\n### Instruction:\n{instruction}\n\n",
    "completion": "{output}",
}
with open("template.json", "w") as f:
    json.dump(template, f)

### Upload dataset to S3
---

I am also uploading the prepared dataset to S3 which will be used for fine-tuning.  **Please note** that during the adaptation of this assignment, I failed to rename the bucket used to upload the JSON file generated from the Medical dictionary.  However, the medical information uploaded correctly as you will see later in the results

---

In [46]:
from sagemaker.s3 import S3Uploader
import sagemaker
import random

output_bucket = sagemaker.Session().default_bucket()
local_data_file = "train.jsonl"
train_data_location = f"s3://{output_bucket}/finbro_dataset"
S3Uploader.upload(local_data_file, train_data_location)
S3Uploader.upload("template.json", train_data_location)
print(f"Training data: {train_data_location}")

Training data: s3://sagemaker-us-west-2-160885283791/finbro_dataset


## Train the model
---

Next, I proceed to train the model with the new information.  I am using a **g5.2xlarge** instance and only conducting 5 epocs

---

In [48]:
from sagemaker.jumpstart.estimator import JumpStartEstimator


estimator = JumpStartEstimator(
    model_id=model_id,
    model_version=model_version,
    environment={"accept_eula": "true"},
    disable_output_compression=True,
    instance_type="ml.g5.2xlarge",  
)
# By default, instruction tuning is set to false. Thus, to use instruction tuning dataset you use
estimator.set_hyperparameters(
    instruction_tuned="True", epoch="5", max_input_length="1024"
)
estimator.fit({"training": train_data_location})

INFO:sagemaker:Creating training-job with name: hf-llm-gemma-2b-2024-10-21-04-28-43-809


2024-10-21 04:28:44 Starting - Starting the training job
2024-10-21 04:28:44 Pending - Training job waiting for capacity...
2024-10-21 04:29:08 Pending - Preparing the instances for training...
2024-10-21 04:29:43 Downloading - Downloading input data............
2024-10-21 04:31:39 Downloading - Downloading the training image..................
2024-10-21 04:34:23 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-10-21 04:34:45,988 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-10-21 04:34:46,007 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-10-21 04:34:46,016 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-10-21 04:34:46,018 sagemaker_pytorch_container.training INFO     Invoking user training script.
2024-10-21 0

### Deploy the fine-tuned model
---
Next, I deploy fine-tuned model, this time to a **g5.xlarge** instance. A comparaison of the performance of fine-tuned and pre-trained model will be done later.

---

In [49]:
finetuned_predictor = estimator.deploy(instance_type="ml.g5.xlarge")

INFO:sagemaker:Creating model with name: hf-llm-gemma-2b-2024-10-21-04-40-41-662
INFO:sagemaker:Creating endpoint-config with name hf-llm-gemma-2b-2024-10-21-04-40-41-661
INFO:sagemaker:Creating endpoint with name hf-llm-gemma-2b-2024-10-21-04-40-41-661


-----------!

### Evaluate the pre-trained and fine-tuned model
---
Next, I adapt the code used in clasee to the test data to evaluate the performance of the fine-tuned model and compare it with the pre-trained model. 

**Please note** I was also experienced some technical difficulties and some errors when using this function, therefore, some code modifications were made.  Also, I left evidence of several "print()" lines to mark the "debugging" I was doing during the code adaptation.  Further, original lines were left commented to mark the difference in reading the different results.

---

In [105]:
import pandas as pd
from IPython.display import display, HTML

test_dataset = train_and_test_dataset["test"]

(
    inputs,
    ground_truth_responses,
    responses_before_finetuning,
    responses_after_finetuning,
) = (
    [],
    [],
    [],
    [],
)


def predict_and_print(datapoint):
    # For instruction fine-tuning, we insert a special key between input and output
    input_output_demarkation_key = "\n\n### Response:\n"

    payload = {
        "inputs": template["prompt"].format(
            instruction=datapoint["instruction"], input=datapoint["input"]
        )
        + input_output_demarkation_key,
        "parameters": {"max_new_tokens": 100},
    }
    
    inputs.append(payload["inputs"])
    #print(payload["inputs"])
    
    ground_truth_responses.append(datapoint["output"])
    
    #print(datapoint["output"])
    
    # Please change the following line to "accept_eula=true"

    #print(payload)
    #print()
    pretrained_response = pretrained_predictor.predict(
        payload, custom_attributes="accept_eula=true"
    )                                                     

    #print(pretrained_response)
    for item in pretrained_response:
        #print (item.get("generated_text"))
        responses_before_finetuning.append(item.get("generated_text"))
        
    # original line.. the 3 lines above substitute this line# responses_before_finetuning.append(pretrained_response.get("generated_text"))
    
    # Fine Tuned Llama 3.2 models doesn't required to set "accept_eula=true"
    #print(payload)
    
    finetuned_response = finetuned_predictor.predict(payload)

    for item in finetuned_response:
        #print (item.get("generated_text"))
        responses_after_finetuning.append(item.get("generated_text"))
    
    # original line.. the 3 lines above substitute this line# responses_after_finetuning.append(finetuned_response.get("generated_text"))

try:
    for i, datapoint in enumerate(test_dataset.select(range(10))):
        predict_and_print(datapoint)
        #print(datapoint)
        #print()

    df = pd.DataFrame(
        {
            "Inputs": inputs,
            "Ground Truth": ground_truth_responses,
            "Response from non-finetuned model": responses_before_finetuning,
            "Response from fine-tuned model": responses_after_finetuning,
        }
    )
    display(HTML(df.to_html()))
except Exception as e:
    print(e)

,Inputs,Ground Truth,Response from non-finetuned model,Response from fine-tuned model
0,"### Input:\nWhat are the stages of Myelodysplastic/ Myeloproliferative Neoplasms ?\n\n### Instruction:\nAnswer the question truthfully, you are a medical professional.\n\n\n\n### Response:\n",Key Points\n - There is no standard staging system for myelodysplastic/myeloproliferative neoplasms.\n \n \n There is no standard staging system for myelodysplastic/myeloproliferative neoplasms.\n Staging is the process used to find out how far the cancer has spread. There is no standard staging system for myelodysplastic /myeloproliferative neoplasms. Treatment is based on the type of myelodysplastic/myeloproliferative neoplasm the patient has. It is important to know the type in order to plan treatment.,"### Input:\nWhat are the stages of Myelodysplastic/ Myeloproliferative Neoplasms ?\n\n### Instruction:\nAnswer the question truthfully, you are a medical professional.\n\n\n\n### Response:\n1. Myelodysplastic/ Myeloproliferative Neoplasms are a group of blood disorders that affect the bone marrow and blood cells. They are characterized by abnormal blood cell production and can lead to anemia, bleeding, and infections.\n\n2. The stages of Myelodysplastic/ Myeloproliferative Neoplasms are:\n\n- Stage 1: This is the earliest stage of the disease, and it is characterized by a low number of blood cells and a","### Input:\nWhat are the stages of Myelodysplastic/ Myeloproliferative Neoplasms ?\n\n### Instruction:\nAnswer the question truthfully, you are a medical professional.\n\n\n\n### Response:\n\n\nThe stages of MDS and MPN are based on the number of abnormal blood cells in the blood and bone marrow. The stages of MDS and MPN are described in the following table:\n\n Stage Description Symptoms Signs Tests and Diagnosis Treatment\n\n I The bone marrow is not working well. There are not enough healthy blood cells, white blood cells, or platelets. The spleen is enlarged. Fatigue, weakness, easy bruising, and bleeding. Swollen"
1,"### Input:\nWhat are the symptoms of Rectal Cancer ?\n\n### Instruction:\nAnswer the question truthfully, you are a medical professional.\n\n\n\n### Response:\n","Signs of rectal cancer include a change in bowel habits or blood in the stool. These and other signs and symptoms may be caused by rectal cancer or by other conditions. Check with your doctor if you have any of the following: - Blood (either bright red or very dark) in the stool. - A change in bowel habits. - Diarrhea. - Constipation. - Feeling that the bowel does not empty completely. - Stools that are narrower or have a different shape than usual. - General abdominal discomfort (frequent gas pains, bloating, fullness, or cramps). - Change in appetite. - Weight loss for no known reason. - Feeling very tired.","### Input:\nWhat are the symptoms of Rectal Cancer ?\n\n### Instruction:\nAnswer the question truthfully, you are a medical professional.\n\n\n\n### Response:\nRectal cancer is a type of cancer that starts in the rectum, the last part of the large intestine. Rectal cancer can spread to other parts of the body. Rectal cancer is the fourth most common cancer in the United States.\n\nSymptoms of rectal cancer include:\n\n- Rectal bleeding\n- Rectal pain\n- Rectal swelling\n- Rectal lump\n- Rectal discharge\n- Rectal weakness\n- Rectal constipation\n- Rectal diarrhea\n- Rectal bleeding","### Input:\nWhat are the symptoms of Rectal Cancer ?\n\n### Instruction:\nAnswer the question truthfully, you are a medical professional.\n\n\n\n### Response:\n.\n\n- Bleeding from the rectum.\n- A lump in the rectum.\n- A change in bowel habits.\n- A change in stool form or consistency.\n- A feeling that the bowel is not emptying completely.\n- A feeling that the bowel is not moving.\n- A feeling of fullness after only a small amount of food.\n- A feeling of nausea or vomiting.\n- A feeling of not being able to control the bowels.\n- A lump in the abdomen"
2,"### Input:\nwhat resear

### Clean up resources

---

After conducting the fine-tuning and model compairason, I went ahead and deleted both model and endpoint for both the pre-trained and fine-tuned model

---

In [106]:
# Delete resources
pretrained_predictor.delete_model()
pretrained_predictor.delete_endpoint()
finetuned_predictor.delete_model()
finetuned_predictor.delete_endpoint()

INFO:sagemaker:Deleting model with name: hf-llm-gemma-2b-2024-10-21-05-07-08-033
INFO:sagemaker:Deleting endpoint configuration with name: hf-llm-gemma-2b-2024-10-21-05-07-08-037
INFO:sagemaker:Deleting endpoint with name: hf-llm-gemma-2b-2024-10-21-05-07-08-037
INFO:sagemaker:Deleting model with name: hf-llm-gemma-2b-2024-10-21-04-40-41-662
INFO:sagemaker:Deleting endpoint configuration with name: hf-llm-gemma-2b-2024-10-21-04-40-41-661
INFO:sagemaker:Deleting endpoint with name: hf-llm-gemma-2b-2024-10-21-04-40-41-661


# Conclusion


---
For the purpose of compairing the results, I limited the scope to 10 results, which was a sufficient number of rows that allowed me to conduct an analysis and draw conclusions.

---

### Observations

- In some instances, the result provided from the pre-trained model is very general and does not offer a high level of detail relative to the original question asked.   In contrast, the fine-tune model, goes into a higher level of detail to anser the question.
  
- In some cases, when the pre-trained model does not have a lot of information, it offers just a reference link.  However, the fine-tuned model once again, offers a higher detailed level in the answer.

- In some cases, the pre-trained model seems to be just paraphrasing the same question (I would argue that the model hallucinates).  In contrast, the fine-tune model continues to offer a detailed explanation, but also provides answers that are coherent and make sense.

- It was evident that in some instances, the pre-trained model is able to provide very limited answers in form of a list, for example, to the question *"Is Blau syndrome inherited?"* the pre-trained model answers **Yes, Blau Syndrome is inherited** . Whereas the fine-tune model would not only provide the correct answer, but it will add more context that made the answer more meaninful and *complete*.  

- With all this being said, it's important to note that a fine-tuned model is not always going to be able to provide the best quality answer.  I was able to observe in some rows that the fine-tuned model was **not able** to provide with an adecuate answer. In some cases the fine-tuned model would *hallucinate*, and in some other cases, the answers will be limited or vague.



### Additional considerations

It is evident, even with such a small sample of answers that the fine-tuned model **outperforms** the pre-trained model. The difference is very evident, even in the case of a **modest** fine-tune effort like the one conducted here.  For reference, the dataset used to fine-tune was only consisting of 2,000 rows and the number of **epocs was only 5**.  

I can say with a high degree of confidence that on a much more *"serious"* effort, with thousands of new medical rows and with a high degree of epocs, the fine-tune model will outperform the original model significantly.


Finally, I would like to say that eventhough the results of this relatively simple fine-tuning exercise were evident and the performance achieved was very significant, it is important to note the **cost** associated with a proper fine-tuning exercise, where we would have to have enough computing resources available or a relatively large budget to outsorce assets to conduct the fine-tuning.    

A cost/benefit analysis of such exercise should always be considered to assess the feasibility and guarantee an acceptable ROI.

  